In [1]:
import numpy as np
from scipy import signal
import tensorflow as tf
import glob
import sys
import os
from timbral_models import filter_audio_highpass, tf_filter_audio_highpass, timbral_util
from timbral_models import timbral_brightness, tf_timbral_brightness_2
from scipy import stats
import warnings
from sklearn.linear_model import LinearRegression
warnings.filterwarnings("ignore")

# Brightness

In [2]:
from timbral_models import timbral_brightness, tf_timbral_brightness_2


data_dir = "/home/ubuntu/Documents/code/data/"
audio_samples, fs = timbral_util.file_read(
    fname, 0, phase_correction=False)
audio_samples2, fs = timbral_util.file_read(
    fname2, 0, phase_correction=False)

tt = 128*128

fps = glob.glob(os.path.join(
    data_dir, "**/*.wav"), recursive=True)
error = []

nn = len(fps)

grad = True
params = []
yy = []
for i, fname in enumerate(fps):
    audio_samples, fs = timbral_util.file_read(
        fname, 0, phase_correction=False)
    audio_samples_t = tf.convert_to_tensor(
        [audio_samples[:tt]], dtype=tf.float32)
    audio_samples_t = tf.expand_dims(audio_samples_t, -1)
    acm_score = np.array(timbral_brightness(fname, dev_output=False))
    yy.append(acm_score)
    tf_score = tf_timbral_brightness_2(
                audio_samples_t, fs=fs, dev_output=True)
    params.append(np.array(tf_score))
    print("File {}/{}".format(i+1, nn), end='\r')
    sys.stdout.flush()
    #error.append(100 * (acm_score - tf_score.numpy()) / acm_score)
print("All done !")
params = np.array(params)[:,:,0]
yy = np.array(yy)

error :: 25.0\%
Tracing with (1, 16384) 20 3 4096 1024 
 y_freq (1, 19, 2049) 
 stft (1, 19, 2049) 
 inverse_stft out shape (1, 16384) 
Tracing with (1, 16384) 20 3 4096 1024 
 y_freq (1, 19, 2049) 
 stft (1, 19, 2049) 
 inverse_stft out shape (1, 16384) 
Tracing with (1, 16384) 2000 3 4096 1024 
 y_freq (1, 19, 2049) 
 stft (1, 19, 2049) 
 inverse_stft out shape (1, 16384) 
Tracing with (1, 16384) 2000 3 4096 1024 
 y_freq (1, 19, 2049) 
 stft (1, 19, 2049) 
 inverse_stft out shape (1, 16384) 
Tracing with (1, 16384) 100 3 4096 1024 
 y_freq (1, 19, 2049) 
 stft (1, 19, 2049) 
 inverse_stft out shape (1, 16384) 
Tracing with (1, 16384) 100 3 4096 1024 
 y_freq (1, 19, 2049) 
 stft (1, 19, 2049) 
 inverse_stft out shape (1, 16384) 


In [6]:
reg = LinearRegression().fit(params, yy)
print("R",reg.score(params, yy))
print("estimated", reg.coef_,reg.intercept_)
print("og",   [4.613128018020465, 17.378889309312974, 17.434733750553022])

coeff 0.9978800316081435 [ 4.5712724 17.355247 ] 17.197018
og [4.613128018020465, 17.378889309312974, 17.434733750553022]


# Booming